In [1]:
import json
import re
import time

import consul
import requests
import sqlalchemy

In [2]:
#FIXME
def _get_service_ip(consul_ip, consul_port, key):
    req_str = "http://" + consul_ip + ":" + str(consul_port) + "/v1/" + key
    #print(req_str)
    res = requests.get(req_str).json()
    for m in res:
        print(json.dumps(m, indent=4))
    return

_get_service_ip("192.168.7.21", 8500, "agent/members")

{
    "Name": "connector-192.168.7.22",
    "Addr": "192.168.7.22",
    "Port": 8301,
    "Tags": {
        "build": "1.0.2:b55059f",
        "dc": "dc1",
        "id": "9742d6e6-d906-93a4-5431-6700663817cb",
        "role": "node",
        "segment": "",
        "vsn": "2",
        "vsn_max": "3",
        "vsn_min": "2"
    },
    "Status": 1,
    "ProtocolMin": 1,
    "ProtocolMax": 5,
    "ProtocolCur": 2,
    "DelegateMin": 2,
    "DelegateMax": 5,
    "DelegateCur": 4
}
{
    "Name": "analytics-engine-192.168.7.25",
    "Addr": "192.168.7.25",
    "Port": 8301,
    "Tags": {
        "build": "1.0.2:b55059f",
        "dc": "dc1",
        "id": "bd5a4e05-2e20-5273-8eaa-4e4c75df30df",
        "role": "node",
        "segment": "",
        "vsn": "2",
        "vsn_max": "3",
        "vsn_min": "2"
    },
    "Status": 1,
    "ProtocolMin": 1,
    "ProtocolMax": 5,
    "ProtocolCur": 2,
    "DelegateMin": 2,
    "DelegateMax": 5,
    "DelegateCur": 4
}
{
    "Name": "collector-192.168.

In [3]:
def consulkv_get_scanner_action_index(consul_ip, scanner_uuid):
    c = consul.Consul(host=consul_ip)
    try:
        res = c.kv.get("scanner/" + scanner_uuid)[1]['Value'].decode()
        #print(res)
        #print(type(res))
        return int(re.findall("(?<=ActionIndex\"\:)(.*?)(?=\,\")", res)[0])
    except TypeError:
        #EXCEPT(e)
        return 0

action_index = consulkv_get_scanner_action_index("192.168.7.11", "HELIOS_SCANNER_27274D56-4370-6BC7-33CC-43D8686E35FF")
print(action_index)

20


In [4]:
def _get_list_from_kv(key):
    c = consul.Consul(host="192.168.7.21")
    try:
        print(c.kv.get(key))
        res = re.sub("\"", "", re.findall("(?<=\"Hashes\"\:\[\")(.*?)(?=\"\])",
                                              c.kv.get(key)[1]['Value'].decode())[0]).split(",")
        return res
    except Exception as e:
        print("Error")
        #EXCEPT(str(e))
        return []

#_get_list_from_kv("locationinfo/192.168.8.74_1433_huaPIITest_pii_1st_uid")

In [5]:
SCANNER_SOURCE_KEY_CONST = ["source/"]
SCANNER_REQUEST_KEY_CONST = ["scanner/", "/action/request/"]
SCANNER_HASHVAL_KEY_CONST = ["locationinfo/"]
SCANNER_DB_MAP = {
    1433: (0, "mssql"),
    3306: (1, "mysql")
}

In [25]:
class KVScannerHandler(object):
    def __init__(self, scan_info_dict):
        #setup info from UI
        self.db_ip = scan_info_dict["ip"]
        self.db_port = scan_info_dict["port"]
        self.db_username = scan_info_dict["username"]
        self.db_password = scan_info_dict["password"]
        self.db_type = scan_info_dict["scan_type"]
        self.badge = scan_info_dict["badge"]
        self.db_tab_col_name = scan_info_dict["selection"]
        self.db_baseline = scan_info_dict["baseline"]
        #setup source uuid and scanner uuid
        self.source_uuid = "_".join([self.db_type, self.db_ip, str(self.db_port)])
        self.scanner_uuid = "HELIOS_SCANNER_74760242-B1F0-7B20-2A62-2EEA7F1CE3AE"
        #hard coded consul
        self.consul_ip = "192.168.7.21"
        self.reqid = self._get_scanner_action_id()
        print("Action id: " + str(self.reqid))
    
    def _get_scanner_action_id(self):
        c = consul.Consul(host=self.consul_ip)
        try:
            res = c.kv.get("scanner/" + self.scanner_uuid)[1]['Value'].decode()
            return int(re.findall("(?<=ActionIndex\"\:)(.*?)(?=\,\")", res)[0])
        except TypeError:
            return 0

    def _get_source_dict(self):
        #try:
        #    scanner_db_type = SCANNER_DB_MAP[self.db_port][0]
        #    scanner_dr_name = SCANNER_DB_MAP[self.db_port][1]
        #except KeyError:
        #    return None
        
        if self.db_baseline and (self.db_type == "MSSQL" or self.db_type == "MySQL"):
            source_scan = {
                "UUID": self.source_uuid,
                "Alias": "sqlcatalog_alias",
                "ScannerID": self.scanner_uuid,
                "TenantID": "helios_tenant_uuid",
                "LastScanTime": "0001-01-01T00:00:00Z",
                "State": 0,
                "RescanInterval": 60000000000,
                "Type": 2,
                "Src": {
                    "UUID": self.source_uuid,
                    "DbInfo": {
                        "Host": self.db_ip,
                        "Port": self.db_port,
                        "Dbtype": scanner_db_type,
                        "Username": self.db_username,
                        "Password": self.db_password,
                        "Dbname": "",
                        "DriverName": scanner_dr_name
                    },
                    "CTVersion": 0
                }
            }
        else:
            if self.db_type == "Samba" or self.db_type == "NFS":
                source_scan = {
                    "UUID": self.source_uuid,
                    "Alias": self.db_type + "_alias",
                    "ScannerID": self.scanner_uuid,
                    "TenantID": "helios_tenant_uuid",
                    "LastScanTime": "0001-01-01T00:00:00Z",
                    "State": 0,
                    "RescanInterval": 60000000000,
                    "Type": 1,
                    "Src": {
                        "FSInfo": {
                            "Host": self.db_ip,
                            "Port": 0,
                            #"Type": 0,
                            "Type": 1,
                            "Username": self.db_username,
                            "Password": self.db_password,
                            "ShareName": "/var/nfs/general"
                            #"ShareName": "sambashare"
                        },
                        "CTVersion": 0,
                        "MountPath": "/opt/helios/filestores/192.168.7.155_/var/nfs/general"
                        #"MountPath": "/opt/helios/filestores/192.168.7.155_sambashare"
                    }
                }
             
            elif self.db_type == "MSSQL" or self.db_type == "MySQL":
                source_scan = {
                    "UUID": self.source_uuid,
                    "Alias": "sqlcontent_alias",
                    "ScannerID": self.scanner_uuid,
                    "TenantID": "helios_tenant_uuid",
                    "LastScanTime": "0001-01-01T00:00:00Z",
                    "State": 0,
                    "RescanInterval": 60000000000,
                    "Type": 0,
                    "DbInfo": {
                        "Host": self.db_ip,
                        "Port": self.db_port,
                        "Dbtype": scanner_db_type,
                        "Username": self.db_username,
                        "Password": self.db_password,
                        "Dbname": "TODO",
                        "DriverName": scanner_dr_name
                    },
                    "Key": {"LastScanTime":"0001-01-01T00:00:00Z", "Table": "TODO", "Column": "eid"},
                    "Tables": [
                        {"TableName": "TODO", "Columns": "TODO"}
                     ],
                    "SigUUID": ""
                }

        return source_scan

    def _scan_db(self):
        c = consul.Consul(host=self.consul_ip)
        #add source
        source_scan = self._get_source_dict()
        if not source_scan:
            return  
        try:
            source_scan_str = json.dumps(source_scan, separators=(',', ':'))
        except TypeError:
            EXCEPT("Unable to dump this source into json string.")
            source_scan_str = ""
        print(source_scan_str)
        print("putting source info into consul")
        c.kv.put(SCANNER_SOURCE_KEY_CONST[0] + self.source_uuid, source_scan_str)
        time.sleep(2)
        #do scan
        print("putting add source request info into consul")
        c.kv.put(SCANNER_REQUEST_KEY_CONST[0] + self.scanner_uuid + SCANNER_REQUEST_KEY_CONST[1] + str(self.reqid),
                 json.dumps({'Action': 3, 'UUID': self.source_uuid}, separators=(',', ':')))
        time.sleep(2)
        self.reqid += 1
        # action to make a on demand scan
        print("putting on demand scan request info into consul") 
        c.kv.put(SCANNER_REQUEST_KEY_CONST[0] + self.scanner_uuid + SCANNER_REQUEST_KEY_CONST[1] + str(self.reqid),
                 json.dumps({'Action': 0, 'UUID': self.source_uuid}, separators=(',', ':')))
        time.sleep(2)
        self.reqid += 1

In [27]:
'''
test_scan_dict = {
    "ip": "192.168.7.155",
    "port": 0,
    "username": "helios",
    "password": "helios123",
    "scan_type": "Samba",
    "badge": "pii",
    "selection": [
        "hr_pii"
    ],
    "baseline": False
}
'''
test_scan_dict = {
    "ip": "192.168.7.155",
    "port": 0,
    "username": "",
    "password": "",
    "scan_type": "NFS",
    "badge": "pii",
    "selection": [
        "hr_pii"
    ],
    "baseline": False
}

myKVScannerHandler = KVScannerHandler(test_scan_dict)
myKVScannerHandler._get_source_dict()
#myKVScannerHandler._scan_db()

Action id: 4


{'Alias': 'NFS_alias',
 'LastScanTime': '0001-01-01T00:00:00Z',
 'RescanInterval': 60000000000,
 'ScannerID': 'HELIOS_SCANNER_74760242-B1F0-7B20-2A62-2EEA7F1CE3AE',
 'Src': {'CTVersion': 0,
  'FSInfo': {'Host': '192.168.7.155',
   'Password': '',
   'Port': 0,
   'ShareName': '/var/nfs/general',
   'Type': 1,
   'Username': ''},
  'MountPath': '/opt/helios/filestores/192.168.7.155_/var/nfs/general'},
 'State': 0,
 'TenantID': 'helios_tenant_uuid',
 'Type': 1,
 'UUID': 'NFS_192.168.7.155_0'}

### End of this file